# 1. 工作线程的清理
如果对象中启动了一个后台工作线程，当线程执行终止前，即使对象变量的作用域已不存在，也是无法调用__del__()(因为线程可能应用到的工作函数是当前对象的一个方法)，而往往我们希望对象过了作用域后可以自动清理，包括终止工作线程，这样就陷入了一种死循环。解决方法是生成一个代理对象，其和线程没有捆绑关系，一旦其变量过了作用域，就会马上调用用其__del__()方法，然后我们在__del__()函数中停止工作线程，这样所有对象都能够正常清理了。

In [3]:
import inspect

class A(object):
    def __init__(self):
        print 'A.__init__() is called'
    def a1(self, x):
        print 'A.a1() is called', x
    def a2(self):
        print 'A.a2() is called'
    def __del__(self):
        print 'A.__del__() is called'

class B(object):
    def __init__(self):
        print 'B.__init__() is call'
        self.__a = A()
        for name, method in inspect.getmembers(self.__a, predicate=inspect.ismethod):
            if name[0] <> '_':
                setattr(self, name, method)
                print name
    def __del__(self):
        print 'B.__del__() is called'
        self.__a = None

b = B()
b.a1('123')
b.a2()
b = None

B.__init__() is call
A.__init__() is called
a1
a2
A.a1() is called 123
A.a2() is called
B.__del__() is called
A.__del__() is called
